In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\heysu\OneDrive\Desktop\essilor_project\data\essilor_chunks_clean.csv")

In [3]:
df.head()

,chunk_id,company,source_file,chunk_text,word_count
0,0,EssilorLuxottica,factset_esg_clean.txt,ESG Report - EssilorLuxottica SA €306.00 ESG O...,544
1,1,EssilorLuxottica,factset_esg_clean.txt,1+2+3 Carbon Intensity Current Year Temperatur...,544
2,0,EssilorLuxottica,factset_financials_clean.txt,EssilorLuxottica SA (EL-FR) Entity Type: Publi...,561
3,1,EssilorLuxottica,factset_financials_clean.txt,29.78 P/FCF 25.09 29.35 40.48 35.19 EssilorLux...,561
4,0,EssilorLuxottica,esg_excerpt_clean.txt,Sustainability Report General Disclosures 1 Ge...,1000


In [4]:
df.columns

Index(['chunk_id', 'company', 'source_file', 'chunk_text', 'word_count'], dtype='object')

In [5]:
len(df)

17

In [6]:
df["source_file"].value_counts()


esg_excerpt_clean.txt              6
annual_report_excerpt_clean.txt    6
factset_esg_clean.txt              2
factset_financials_clean.txt       2
external_summary_clean.txt         1
Name: source_file, dtype: int64

In [7]:
import sys, pathlib
sys.path.append(str(pathlib.Path("C:/Users/heysu/OneDrive/Desktop/essilor_project/src")))

from document_store import build_document_store

df_norm = build_document_store()



In [8]:
df_norm.head()

,chunk_id,text,source,doc_type,year
0,0,ESG Report - EssilorLuxottica SA €306.00 ESG O...,factset_esg_clean.txt,esg,2024
1,1,1+2+3 Carbon Intensity Current Year Temperatur...,factset_esg_clean.txt,esg,2024
2,0,EssilorLuxottica SA (EL-FR) Entity Type: Publi...,factset_financials_clean.txt,financial,2024
3,1,29.78 P/FCF 25.09 29.35 40.48 35.19 EssilorLux...,factset_financials_clean.txt,financial,2024
4,0,Sustainability Report General Disclosures 1 Ge...,esg_excerpt_clean.txt,esg,2024


In [9]:
df_norm.columns

Index(['chunk_id', 'text', 'source', 'doc_type', 'year'], dtype='object')

In [10]:
df_norm["doc_type"].value_counts()

esg          8
annual       6
financial    2
news         1
Name: doc_type, dtype: int64

In [11]:
df_norm["year"].value_counts()

2024    17
Name: year, dtype: int64

In [12]:
len(df_norm)


17

In [13]:
import sys, pathlib
sys.path.append(str(pathlib.Path("C:/Users/heysu/OneDrive/Desktop/essilor_project/src")))

from retrieval import build_tfidf_artifacts

build_tfidf_artifacts()


Saved TF-IDF artifacts: 17 chunks, matrix shape (17, 10609)


In [14]:
from retrieval import retrieve_chunks

# ESG-focused question
q1 = "What are EssilorLuxottica's total greenhouse gas emissions and Scope 1, 2, and 3 breakdown?"
res1 = retrieve_chunks(q1, k=5, allowed_doc_types=["esg"])
res1[["chunk_id", "source", "doc_type", "similarity"]]


,chunk_id,source,doc_type,similarity
0,0,factset_esg_clean.txt,esg,0.113095
1,1,factset_esg_clean.txt,esg,0.041232
2,2,esg_excerpt_clean.txt,esg,0.040445
3,5,esg_excerpt_clean.txt,esg,0.019436
4,4,esg_excerpt_clean.txt,esg,0.010889


In [15]:
# Financial question
q2 = "What was EssilorLuxottica's revenue and EBITDA in 2024?"
res2 = retrieve_chunks(q2, k=5, allowed_doc_types=["financial", "annual"])
res2[["chunk_id", "source", "doc_type", "similarity"]]


,chunk_id,source,doc_type,similarity
0,2,annual_report_excerpt_clean.txt,annual,0.091580
1,0,factset_financials_clean.txt,financial,0.062270
2,1,annual_report_excerpt_clean.txt,annual,0.021910
3,0,annual_report_excerpt_clean.txt,annual,0.014550
4,1,factset_financials_clean.txt,financial,0.014528


In [16]:
pip install openai


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip show openai


Name: openai
Version: 2.8.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: c:\Users\heysu\anaconda3\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [18]:
import openai
openai.__version__


'2.8.1'

In [19]:
from llm_agent import answer_question

answer_question(
    "What are EssilorLuxottica's Scope 1, 2, and 3 emissions?",
    allowed_doc_types=["esg"]
)

{'answer': "1. EssilorLuxottica's total greenhouse gas emissions for FY24 amount to 4,119,954 tCO2e. These are broken down into Scope 1 emissions of 116,092 tCO2e, Scope 2 emissions of 475,555 tCO2e, and Scope 3 emissions of 3,528,307 tCO2e. The company reports these emissions alongside intensity metrics per EUR million EVIC, with Scope 1 intensity at 0.9499, Scope 2 at 3.8909, and Scope 3 at 28.8683 tCO2e/EUR million EVIC [chunk_id=0]. The emissions data is aligned with the 2024 reporting and reflects the company’s commitment to tracking its carbon footprint as part of its Eyes on the Planet sustainability program [chunk_id=2].\n\n2. The provided documents do not specify the methodologies or boundaries used for calculating Scope 3 emissions in detail, nor do they provide historical emissions data or targets for reduction beyond mentioning a 2030 target in general terms.\n\n3. chunk_ids used: 0, 2",
 'chunks':    chunk_id                                               text  \
 0        

In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path("C:/Users/heysu/OneDrive/Desktop/essilor_project/src")))

from kpis import get_kpis_df

get_kpis_df()
get_kpis_df("financial")
get_kpis_df("esg")


,name,category,value,unit,year,description,source,chunk_ids,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954,tCO2e,2024,Total greenhouse gas emissions for FY24 across...,factset_esg_clean.txt,0,"Sum of Scope 1 (116,092), Scope 2 (475,555) an..."
1,Scope 1 emissions,esg,116092,tCO2e,2024,Direct Scope 1 emissions from owned or control...,factset_esg_clean.txt,0,Intensity: 0.9499 tCO2e per EUR million EVIC.
2,Scope 2 emissions (market-based),esg,475555,tCO2e,2024,Scope 2 emissions from purchased energy (marke...,factset_esg_clean.txt,0,Intensity: 3.8909 tCO2e per EUR million EVIC.
3,Scope 3 emissions,esg,3528307,tCO2e,2024,Scope 3 value-chain emissions (indirect).,factset_esg_clean.txt,0,Intensity: 28.8683 tCO2e per EUR million EVIC.


In [23]:
from llm_agent import answer_question

res_1 = answer_question("What are EssilorLuxottica's Scope 1 emissions?", allowed_doc_types=["esg"])
res_2 = answer_question("What are EssilorLuxottica's Scope 2 emissions?", allowed_doc_types=["esg"])
res_3 = answer_question("What are EssilorLuxottica's Scope 3 emissions?", allowed_doc_types=["esg"])

print("SCOPE 1:\n", res_1["answer"], "\n")
print("SCOPE 2:\n", res_2["answer"], "\n")
print("SCOPE 3:\n", res_3["answer"], "\n")


SCOPE 1:
 1. EssilorLuxottica's Scope 1 emissions for the fiscal year 2024 are reported as 116,092 tonnes of CO2 equivalent (tCO2e). This figure represents the direct greenhouse gas emissions from sources owned or controlled by the company. The total emissions including Scope 1, 2, and 3 amount to 4,119,954 tCO2e, with Scope 1 being a relatively small portion of the total footprint. The company does not currently have carbon emission reduction initiatives aimed at aligning with the Paris Agreement.  

2. The information on Scope 1 emissions is clearly stated in the 2024 ESG data; however, details on the breakdown of sources or trends over time are not provided in the available documents.

3. chunk_ids used: [0] 

SCOPE 2:
 1. EssilorLuxottica's Scope 2 emissions for FY24 are reported as 475,555 tCO2e. This figure represents the greenhouse gas emissions from purchased energy sources. The total emissions for the company amount to 4,119,954 tCO2e, with Scope 1 emissions at 116,092 tCO2e a

In [22]:
import importlib, kpis
importlib.reload(kpis)

from kpis import get_kpis_df

get_kpis_df("esg")


,name,category,value,unit,year,description,source,chunk_ids,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954,tCO2e,2024,Total greenhouse gas emissions for FY24 across...,factset_esg_clean.txt,0,"Sum of Scope 1 (116,092), Scope 2 (475,555) an..."
1,Scope 1 emissions,esg,116092,tCO2e,2024,Direct Scope 1 emissions from owned or control...,factset_esg_clean.txt,0,Intensity: 0.9499 tCO2e per EUR million EVIC.
2,Scope 2 emissions (market-based),esg,475555,tCO2e,2024,Scope 2 emissions from purchased energy (marke...,factset_esg_clean.txt,0,Intensity: 3.8909 tCO2e per EUR million EVIC.
3,Scope 3 emissions,esg,3528307,tCO2e,2024,Scope 3 value-chain emissions (indirect).,factset_esg_clean.txt,0,Intensity: 28.8683 tCO2e per EUR million EVIC.


In [1]:
from llm_agent import answer_question

res_1 = answer_question("What are EssilorLuxottica's Scope 1 emissions?", allowed_doc_types=["esg"])
res_2 = answer_question("What are EssilorLuxottica's Scope 2 market-based emissions?", allowed_doc_types=["esg"])
res_3 = answer_question("What was EBITDA for FY24", allowed_doc_types=["esg"])

print("SCOPE 1:\n", res_1["answer"], "\n")
print("SCOPE 2:\n", res_2["answer"], "\n")
print("SCOPE 3:\n", res_3["answer"], "\n")


ModuleNotFoundError: No module named 'llm_agent'

In [2]:
import pandas as pd
from pathlib import Path

csv_path = Path(r"C:\Users\heysu\OneDrive\Desktop\essilor_project\data\kpis_auto.csv")
df = pd.read_csv(csv_path)

df[["name", "value", "unit", "confidence", "reason", "raw_snippet"]]


,name,value,unit,confidence,reason,raw_snippet
0,Total GHG emissions (Scope 1+2+3),4119954.0,tCO2e,high,"Chunk 0 provides exact total Scope 1, 2 and 3 ...",Key Items Metric FY24 Emissions Emissions (tCO...
1,Scope 1 emissions,NaN,tCO2e,low,Failed to parse JSON. Raw response not valid J...,NaN
2,Scope 2 emissions (market-based),NaN,tCO2e,low,Failed to parse JSON. Raw response not valid J...,NaN
3,Scope 3 emissions,3528307.0,tCO2e,high,Chunk 0 explicitly states EssilorLuxottica's S...,"Scope 3 emissions 3,528,307.00"


In [3]:
df = pd.read_csv(csv_path)
df[["name", "value", "unit", "confidence", "reason", "raw_snippet"]]


,name,value,unit,confidence,reason,raw_snippet
0,Total GHG emissions (Scope 1+2+3),4119954.0,tCO2e,high,Chunk 0 provides exact total GHG emissions for...,Key Items Metric FY24 Emissions Emissions (tCO...
1,Scope 1 emissions,NaN,tCO2e,low,Failed to parse JSON. Raw response not valid J...,NaN
2,Scope 2 emissions (market-based),NaN,tCO2e,low,Failed to parse JSON. Raw response not valid J...,NaN
3,Scope 3 emissions,3528307.0,tCO2e,high,Exact Scope 3 emissions value for FY24 is expl...,"Scope 3 emissions 3,528,307.00"


In [4]:
import sys, pathlib
sys.path.append(str(pathlib.Path("C:/Users/heysu/OneDrive/Desktop/essilor_project/src")))

import importlib, kpis
importlib.reload(kpis)

from kpis import get_kpis_df

get_kpis_df("esg")


,name,category,value,unit,year,description,source,chunk_ids,confidence,reason,raw_snippet,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954.0,tCO2e,2024,"What are EssilorLuxottica's total Scope 1, 2 a...","factset_esg_clean.txt, esg_excerpt_clean.txt",,high,"Chunk 0 provides exact total Scope 1, 2 and 3 ...",Key Items Metric FY24 Emissions Emissions (tCO...,
1,Scope 1 emissions,esg,116092.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
2,Scope 2 emissions (market-based),esg,475555.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
3,Scope 3 emissions,esg,3528307.0,tCO2e,2024,What are EssilorLuxottica's Scope 3 emissions ...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,high,Exact Scope 3 emissions value for FY24 is expl...,"Scope 3 emissions 3,528,307.00",


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
from kpis import get_kpis_df
get_kpis_df()          # all
get_kpis_df("financial")
get_kpis_df("esg")


,name,category,value,unit,year,description,source,chunk_ids,confidence,reason,raw_snippet,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954.0,tCO2e,2024,"What are EssilorLuxottica's total Scope 1, 2 a...","factset_esg_clean.txt, esg_excerpt_clean.txt",,high,"Chunk 0 provides exact total Scope 1, 2 and 3 ...",Key Items Metric FY24 Emissions Emissions (tCO...,
1,Scope 1 emissions,esg,116092.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
2,Scope 2 emissions (market-based),esg,475555.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
3,Scope 3 emissions,esg,3528307.0,tCO2e,2024,What are EssilorLuxottica's Scope 3 emissions ...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,high,Exact Scope 3 emissions value for FY24 is expl...,"Scope 3 emissions 3,528,307.00",


In [7]:
from kpis import get_kpis_df

In [8]:
get_kpis_df()          

,name,category,value,unit,year,description,source,chunk_ids,confidence,reason,raw_snippet,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954.0,tCO2e,2024,"What are EssilorLuxottica's total Scope 1, 2 a...","factset_esg_clean.txt, esg_excerpt_clean.txt",,high,"Chunk 0 provides exact total Scope 1, 2 and 3 ...",Key Items Metric FY24 Emissions Emissions (tCO...,
1,Scope 1 emissions,esg,116092.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
2,Scope 2 emissions (market-based),esg,475555.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
3,Scope 3 emissions,esg,3528307.0,tCO2e,2024,What are EssilorLuxottica's Scope 3 emissions ...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,high,Exact Scope 3 emissions value for FY24 is expl...,"Scope 3 emissions 3,528,307.00",


In [9]:
get_kpis_df("financial")

""


In [10]:
get_kpis_df("esg")

,name,category,value,unit,year,description,source,chunk_ids,confidence,reason,raw_snippet,notes
0,Total GHG emissions (Scope 1+2+3),esg,4119954.0,tCO2e,2024,"What are EssilorLuxottica's total Scope 1, 2 a...","factset_esg_clean.txt, esg_excerpt_clean.txt",,high,"Chunk 0 provides exact total Scope 1, 2 and 3 ...",Key Items Metric FY24 Emissions Emissions (tCO...,
1,Scope 1 emissions,esg,116092.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
2,Scope 2 emissions (market-based),esg,475555.0,tCO2e,2024,What is the numeric value of EssilorLuxottica'...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,manual,Manually set from verified ESG figures.,NaN,
3,Scope 3 emissions,esg,3528307.0,tCO2e,2024,What are EssilorLuxottica's Scope 3 emissions ...,"factset_esg_clean.txt, esg_excerpt_clean.txt",,high,Exact Scope 3 emissions value for FY24 is expl...,"Scope 3 emissions 3,528,307.00",
